In [3]:
from dataset import normalizate
norm = normalizate('datas/exemplo', "datas/exemplosnormalizados")
norm.preprocess_and_segment_audio('datas/exemplo/exemplo.mp3',"datas/exemplosnormalizados" )

In [1]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
from wakeword import WakeWordCNN

print("Iniciando o modo de inferência do modelo de Wake Word...")
from joblib import load
scaler = load("scaler.pkl")
SAMPLERATE = 16000
SEGMENT_DURATION_SECONDS = 3
N_MFCC = 40     
N_FFT = 400       
HOP_LENGTH = 160  
SCALER_LOAD_PATH = "scaler.pkl"
scaler = load(SCALER_LOAD_PATH)
MODEL_LOAD_PATH = "models/wake_word_model.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")








def predict_wake_word(audio_filepath, model, mfcc_transform, scaler, segment_duration_seconds, samplerate, device, threshold):
    """
    Realiza a inferência em um arquivo de áudio para detectar a wake word.
    """
    model.eval() # Coloca o modelo em modo de avaliação

    # Pré-processa o áudio
    mfccs = preprocess_audio_for_inference(audio_filepath, mfcc_transform, scaler, segment_duration_seconds, samplerate)
    if mfccs is None:
        return None, None # Retorna None se houver erro no pré-processamento

    # Adiciona uma dimensão de lote (batch dimension) pois o modelo espera um lote
    # mfccs.shape de (1, N_MFCC, NUM_FRAMES_PER_SAMPLE) para (1, 1, N_MFCC, NUM_FRAMES_PER_SAMPLE)
    mfccs = mfccs.unsqueeze(0).to(device)

    with torch.no_grad(): # Desativa o cálculo de gradientes para inferência
        outputs = model(mfccs)
        probability = torch.sigmoid(outputs).item() # Converte logits para probabilidade (0 a 1)

    prediction = 1 if probability >= threshold else 0 # 1 para wake word, 0 para não wake word

    return prediction, probability


def preprocess_audio_for_inference(audio_filepath, mfcc_transform, scaler, segment_duration_seconds, samplerate):
    
    """
    Carrega, pré-processa e extrai MFCCs de um único arquivo de áudio para inferência.
    """
    try:
        waveform, sr = torchaudio.load(audio_filepath)

        # 1. Converte para mono
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # 2. Resample para a taxa de amostragem correta
        if sr != samplerate:
            resampler = torchaudio.transforms.Resample(sr, samplerate)
            waveform = resampler(waveform)

        # 3. Preenche ou corta para a duração esperada
        expected_num_samples = int(samplerate * segment_duration_seconds)
        if waveform.shape[1] < expected_num_samples:
            padding = expected_num_samples - waveform.shape[1]
            waveform = F.pad(waveform, (0, padding))
        elif waveform.shape[1] > expected_num_samples:
            waveform = waveform[:, :expected_num_samples]

        # 4. Extrai MFCCs
        mfccs = mfcc_transform(waveform)

        # 5. Normaliza MFCCs usando o scaler carregado
        mfccs_np = mfccs.squeeze(0).numpy()
        mfccs_normalized = scaler.transform(mfccs_np.T).T # Transpõe, normaliza, transpõe de volta
        mfccs = torch.from_numpy(mfccs_normalized).float().unsqueeze(0) # Adiciona dimensão de lote e canal

        return mfccs

    except FileNotFoundError:
        print(f"Erro: Arquivo de áudio não encontrado em '{audio_filepath}'.")
        return None
    except Exception as e:
        print(f"Erro durante o pré-processamento do áudio: {e}")
        return None



# Calcula o número de frames de MFCC por amostra de áudio (DEVE ser o mesmo do treinamento)
dummy_waveform = torch.randn(1, int(SAMPLERATE * SEGMENT_DURATION_SECONDS))
mfcc_test_transform = torchaudio.transforms.MFCC(
    sample_rate=SAMPLERATE, n_mfcc=N_MFCC,
    melkwargs={"n_fft": N_FFT, "hop_length": HOP_LENGTH, "n_mels": N_MFCC}
)
dummy_mfccs = mfcc_test_transform(dummy_waveform)
NUM_FRAMES_PER_SAMPLE = dummy_mfccs.shape[2]
print(f"Número de frames MFCC por amostra: {NUM_FRAMES_PER_SAMPLE}")

# 1. Carrega o scaler
try:
    print(f"Scaler carregado de: {SCALER_LOAD_PATH}")
except FileNotFoundError:
    print(f"Erro: Scaler não encontrado em '{SCALER_LOAD_PATH}'. Certifique-se de ter treinado o modelo e salvo o scaler.")
    exit()
except Exception as e:
    print(f"Erro ao carregar o scaler: {e}")
    exit()

# 2. Instancia o modelo e carrega os pesos treinados
model = WakeWordCNN(N_MFCC, NUM_FRAMES_PER_SAMPLE).to(DEVICE)
try:
    model.load_state_dict(torch.load(MODEL_LOAD_PATH, map_location=DEVICE))
    print(f"Modelo carregado de: {MODEL_LOAD_PATH}")
except FileNotFoundError:
    print(f"Erro: Modelo não encontrado em '{MODEL_LOAD_PATH}'. Certifique-se de ter treinado o modelo.")
    exit()
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")
    exit()

# 3. Inicializa o transformador MFCC (o mesmo usado no Dataset)
mfcc_transform_inference = torchaudio.transforms.MFCC(
    sample_rate=SAMPLERATE,
    n_mfcc=N_MFCC,
    melkwargs={
        "n_fft": N_FFT,
        "hop_length": HOP_LENGTH,
        "n_mels": N_MFCC
    }
)

PREDICTION_THRESHOLD = 0.5
# --- Teste com um arquivo de áudio ---
# ALtere este caminho para o seu arquivo de áudio de teste!
# Pode ser um áudio da sua pasta 'positive/' ou 'negative/'
TEST_AUDIO_FILE = "datas/exemplosnormalizados/exemplo_segment_0001.wav" # <-- ALtere este caminho!
# Ou um áudio da sua pasta 'negative_processed/'
# TEST_AUDIO_FILE = "wake_word_dataset/negative_processed/some_negative_audio_segment_0001.wav"

if not os.path.exists(TEST_AUDIO_FILE):
    print(f"\nAVISO: O arquivo de áudio de teste '{TEST_AUDIO_FILE}' não foi encontrado.")
    print("Por favor, altere a variável 'TEST_AUDIO_FILE' no script para um caminho válido.")
    print("Você pode usar um dos arquivos que você gravou na pasta 'wake_word_dataset/positive' ou 'wake_word_dataset/negative'.")
else:
    print(f"\nRealizando inferência no arquivo: {TEST_AUDIO_FILE}")
    prediction, probability = predict_wake_word(
        TEST_AUDIO_FILE,
        model,
        mfcc_transform_inference,
        scaler,
        SEGMENT_DURATION_SECONDS,
        SAMPLERATE,
        DEVICE,
        PREDICTION_THRESHOLD
    )

    if prediction is not None:
        print(f"Probabilidade da Wake Word: {probability:.4f}")
        if prediction == 1:
            print("Resultado: É a WAKE WORD!")
        else:
            print("Resultado: NÃO é a Wake Word.")
    else:
        print("Não foi possível realizar a inferência.")

print("\nInferência concluída.")


Iniciando o modo de inferência do modelo de Wake Word...
Número de frames MFCC por amostra: 301
Scaler carregado de: scaler.pkl
Modelo carregado de: models/wake_word_model.pth

Realizando inferência no arquivo: datas/exemplosnormalizados/exemplo_segment_0001.wav
Probabilidade da Wake Word: 0.9876
Resultado: É a WAKE WORD!

Inferência concluída.
